## 1 - not using gdal but with memory issue

In [ ]:
import rasterio
from rasterio.merge import merge
import glob
import os

# Folder containing your NDVI .tif files
input_folder = "G:/My Drive/GEE_Exports"
tif_files = glob.glob(os.path.join(input_folder, "*.tif"))

# Open all rasters
src_files_to_mosaic = [rasterio.open(tif) for tif in tif_files]

# Merge them into one array
mosaic, out_transform = merge(src_files_to_mosaic)

# Scale NDVI (multiply by 100) and convert to Int16
mosaic_scaled = (mosaic * 100).astype("int16")

# Copy metadata and update
out_meta = src_files_to_mosaic[0].meta.copy()
out_meta.update({
    "driver": "GTiff",
    "height": mosaic.shape[1],
    "width": mosaic.shape[2],
    "transform": out_transform,
    "dtype": "int16",       # Save as Int16
    "compress": "LZW",      # Add LZW compression
    "count": mosaic.shape[0]
})

# Save scaled and compressed mosaic
output_file = "D:/NDVI_median_landsat_30m_2021_USA_mosaic.tif"
with rasterio.open(output_file, "w", **out_meta) as dest:
    dest.write(mosaic_scaled)

# Close input files
for src in src_files_to_mosaic:
    src.close()


## 2 - using gdal to solve memory issue

In [6]:
import rasterio
from rasterio.enums import Resampling
from rasterio.shutil import copy as rio_copy
from rasterio.transform import Affine
import glob
import os
from osgeo import gdal

# Paths
input_folder = r"G:/My Drive/GEE_Exports"
output_folder = r"D:/NDVI_output"
os.makedirs(output_folder, exist_ok=True)

# Find all .tif files
tif_files = glob.glob(os.path.join(input_folder, "*.tif"))
print("TIF files found:", len(tif_files))

if not tif_files:
    raise RuntimeError("No .tif files found in input folder.")

# Step 1: Scale each tile (NDVI * 100) with Rasterio
scaled_files = []

for tif in tif_files:
    with rasterio.open(tif) as src:
        profile = src.profile
        profile.update(
            dtype='int16',
            compress='lzw',
            tiled=True,
            predictor=2,
            count=src.count  # keep band count
        )

        scaled_file = os.path.join(output_folder, f"scaled_{os.path.basename(tif)}")
        print("Scaling and saving:", scaled_file)

        with rasterio.open(scaled_file, 'w', **profile) as dst:
            for i in range(1, src.count + 1):
                band = src.read(i)
                scaled_band = (band * 100).astype('int16')
                dst.write(scaled_band, i)
        scaled_files.append(scaled_file)

print("All tiles scaled and saved.")

# Step 2: Mosaic scaled tiles with GDAL
vrt_file = os.path.join(output_folder, "mosaic.vrt")
mosaic_file = os.path.join(output_folder, "NDVI_median_landsat_30m_2021_USA_mosaic.tif")

# Build VRT
vrt_ds = gdal.BuildVRT(vrt_file, scaled_files)
vrt_ds.FlushCache()
vrt_ds = None
print(f"VRT created: {vrt_file}")

# Translate VRT to compressed GeoTIFF
gdal.Translate(
    mosaic_file,
    vrt_file,
    creationOptions=["COMPRESS=LZW", "PREDICTOR=2", "TILED=YES"],
    outputType=gdal.GDT_Int16
)
print(f"Mosaic saved to: {mosaic_file}")

# Clean up VRT (optional)
os.remove(vrt_file)
print("Temporary VRT removed.")


TIF files found: 57
Scaling and saving: D:/NDVI_output\scaled_NDVI_median_landsat_30m_29_2021.tif


C:\Users\pc\AppData\Local\Temp\ipykernel_5504\2473876304.py:41: RuntimeWarning: invalid value encountered in cast
  scaled_band = (band * 100).astype('int16')


Scaling and saving: D:/NDVI_output\scaled_NDVI_median_landsat_30m_31_2021.tif
Scaling and saving: D:/NDVI_output\scaled_NDVI_median_landsat_30m_27_2021.tif
Scaling and saving: D:/NDVI_output\scaled_NDVI_median_landsat_30m_20_2021.tif
Scaling and saving: D:/NDVI_output\scaled_NDVI_median_landsat_30m_46_2021.tif
Scaling and saving: D:/NDVI_output\scaled_NDVI_median_landsat_30m_38_2021.tif
Scaling and saving: D:/NDVI_output\scaled_NDVI_median_landsat_30m_54_2021.tif
Scaling and saving: D:/NDVI_output\scaled_NDVI_median_landsat_30m_19_2021.tif
Scaling and saving: D:/NDVI_output\scaled_NDVI_median_landsat_30m_12_2021.tif
Scaling and saving: D:/NDVI_output\scaled_NDVI_median_landsat_30m_10_2021.tif
Scaling and saving: D:/NDVI_output\scaled_NDVI_median_landsat_30m_37_2021-0000000000-0000000000.tif
Scaling and saving: D:/NDVI_output\scaled_NDVI_median_landsat_30m_13_2021.tif
Scaling and saving: D:/NDVI_output\scaled_NDVI_median_landsat_30m_11_2021.tif
Scaling and saving: D:/NDVI_output\scaled_

In [7]:
# ====== Step 2: Mosaic scaled tiles with GDAL ======
# Build VRT
vrt_file = os.path.join(output_folder, "mosaic.vrt")
vrt_ds = gdal.BuildVRT(vrt_file, scaled_files)
if vrt_ds is None:
    raise RuntimeError("Failed to build VRT.")
vrt_ds.FlushCache()  # Force write VRT to disk
vrt_ds = None
print(f"VRT created and saved: {vrt_file}")

# Translate VRT to compressed GeoTIFF
translate_ds = gdal.Translate(
    mosaic_file,
    vrt_file,
    creationOptions=["COMPRESS=LZW", "PREDICTOR=2", "TILED=YES"],
    outputType=gdal.GDT_Int16
)
if translate_ds is None:
    raise RuntimeError("Failed to save mosaic.")
translate_ds.FlushCache()  # Force write to disk
translate_ds = None
print(f"Mosaic saved to: {mosaic_file}")

# ====== Optional: Clean up VRT ======
if os.path.exists(vrt_file):
    os.remove(vrt_file)
    print("Temporary VRT removed.")

# ====== Final Check ======
if os.path.exists(mosaic_file):
    print("✅ Mosaic successfully saved:", mosaic_file)
else:
    raise RuntimeError("❌ Mosaic file was not created!")

VRT created and saved: D:/NDVI_output\mosaic.vrt


RuntimeError: Failed to save mosaic.

## 3 - deepseek 

In [8]:
import rasterio
from rasterio.enums import Resampling
import glob
import os
from osgeo import gdal
import numpy as np
import time

# ====== Initialize GDAL exceptions ======
gdal.UseExceptions()

# ====== Configuration ======
INPUT_FOLDER = r"G:/My Drive/GEE_Exports"
OUTPUT_FOLDER = r"D:/NDVI_output"
MOSAIC_NAME = "NDVI_median_landsat_30m_2021_USA_mosaic.tif"
COMPRESSION_OPTS = {
    'compress': 'lzw',
    'predictor': 2,
    'tiled': True,
    'blockxsize': 256,
    'blockysize': 256
}

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ====== Helper Functions ======
def validate_geotiff(file_path):
    """Basic validation of GeoTIFF file"""
    try:
        with rasterio.open(file_path) as src:
            if src.width == 0 or src.height == 0:
                return False
            data = src.read(1, window=((0,1), (0,1)))
        return os.path.getsize(file_path) > 1024  # At least 1KB
    except:
        return False

# ====== Step 1: Scale NDVI tiles with Rasterio ======
tif_files = sorted(glob.glob(os.path.join(INPUT_FOLDER, "*.tif")))
print(f"Found {len(tif_files)} TIFF files")

if not tif_files:
    raise RuntimeError("No input files found")

scaled_files = []
processing_errors = []

for idx, tif_path in enumerate(tif_files, 1):
    try:
        base_name = os.path.basename(tif_path)
        output_path = os.path.join(OUTPUT_FOLDER, f"scaled_{base_name}")
        
        if os.path.exists(output_path) and validate_geotiff(output_path):
            print(f"Skipping existing valid file ({idx}/{len(tif_files)}): {base_name}")
            scaled_files.append(output_path)
            continue

        with rasterio.open(tif_path) as src:
            # Update profile with compression and data type
            profile = src.profile.copy()
            profile.update({
                'dtype': 'int16',
                'driver': 'GTiff',
                **COMPRESSION_OPTS
            })

            # Scale and write data
            with rasterio.open(output_path, 'w', **profile) as dst:
                for band_idx in range(1, src.count + 1):
                    data = src.read(band_idx)
                    scaled_data = (data * 100).astype('int16')
                    dst.write(scaled_data, band_idx)

                # Force write to disk
                dst.close()
                os.fsync(dst.fileno())

        # Validate output
        if validate_geotiff(output_path):
            scaled_files.append(output_path)
            print(f"Processed ({idx}/{len(tif_files)}): {base_name}")
        else:
            raise RuntimeError("Output file validation failed")

    except Exception as e:
        processing_errors.append(f"{tif_path}: {str(e)}")
        print(f"Error processing {base_name}: {str(e)}")

if processing_errors:
    raise RuntimeError(f"{len(processing_errors)} errors occurred:\n" + "\n".join(processing_errors))

print(f"Successfully processed {len(scaled_files)}/{len(tif_files)} files")

# ====== Step 2: Create Mosaic with GDAL ======
vrt_path = os.path.join(OUTPUT_FOLDER, "mosaic.vrt")
mosaic_path = os.path.join(OUTPUT_FOLDER, MOSAIC_NAME)

try:
    # Build VRT
    print("Building VRT mosaic...")
    vrt_options = gdal.BuildVRTOptions(resampleAlg='bilinear', addAlpha=False)
    vrt_ds = gdal.BuildVRT(vrt_path, scaled_files, options=vrt_options)
    vrt_ds = None  # Flush VRT to disk

    # Convert to compressed GeoTIFF
    print("Creating final mosaic...")
    translate_options = gdal.TranslateOptions(
        creationOptions=[
            'COMPRESS=LZW',
            'PREDICTOR=2',
            'TILED=YES',
            'BIGTIFF=IF_SAFE',
            'BLOCKXSIZE=256',
            'BLOCKYSIZE=256'
        ],
        outputType=gdal.GDT_Int16
    )

    gdal.Translate(mosaic_path, vrt_path, options=translate_options)
    
    # Final validation
    if not validate_geotiff(mosaic_path):
        raise RuntimeError("Final mosaic validation failed")

except Exception as e:
    if os.path.exists(mosaic_path):
        os.remove(mosaic_path)
    raise RuntimeError(f"Mosaic creation failed: {str(e)}")

finally:
    # Cleanup temporary files
    if os.path.exists(vrt_path):
        os.remove(vrt_path)

# ====== Final Report ======
print("\n" + "="*50)
print(f"Successfully created mosaic: {mosaic_path}")
print(f"Output size: {os.path.getsize(mosaic_path)/1024/1024:.1f} MB")
print("="*50 + "\n")

Found 57 TIFF files
Skipping existing valid file (1/57): NDVI_median_landsat_30m_01_2021.tif
Skipping existing valid file (2/57): NDVI_median_landsat_30m_04_2021.tif
Skipping existing valid file (3/57): NDVI_median_landsat_30m_05_2021.tif
Skipping existing valid file (4/57): NDVI_median_landsat_30m_06_2021-0000000000-0000000000.tif
Skipping existing valid file (5/57): NDVI_median_landsat_30m_06_2021-0000000000-0000032768.tif
Skipping existing valid file (6/57): NDVI_median_landsat_30m_06_2021-0000032768-0000000000.tif
Skipping existing valid file (7/57): NDVI_median_landsat_30m_06_2021-0000032768-0000032768.tif
Skipping existing valid file (8/57): NDVI_median_landsat_30m_08_2021.tif
Skipping existing valid file (9/57): NDVI_median_landsat_30m_09_2021.tif
Skipping existing valid file (10/57): NDVI_median_landsat_30m_10_2021.tif
Skipping existing valid file (11/57): NDVI_median_landsat_30m_11_2021.tif
Skipping existing valid file (12/57): NDVI_median_landsat_30m_12_2021.tif
Skipping exis

## 4 - perplexity

In [6]:
import rasterio
import glob
import os
from osgeo import gdal
import numpy as np

# ====== Configuration ======
INPUT_FOLDER = r"G:/My Drive/GEE_Exports"     # Folder containing original NDVI tiles
OUTPUT_FOLDER = r"D:/NDVI_output"             # Folder to store scaled tiles and mosaic
MOSAIC_NAME = "NDVI_median_landsat_30m_2021_USA_mosaic_v1.tif"
COMPRESSION_OPTS = [
    'COMPRESS=LZW',
    'PREDICTOR=2',
    'TILED=YES',
    'BIGTIFF=IF_SAFE',
    'BLOCKXSIZE=256',
    'BLOCKYSIZE=256'
]

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ====== Helper Functions ======
def validate_geotiff(file_path):
    try:
        with rasterio.open(file_path) as src:
            if src.width == 0 or src.height == 0:
                return False
            data = src.read(1, window=((0,1), (0,1)))
        return os.path.getsize(file_path) > 1024
    except Exception:
        return False

# ====== Step 1: Scale NDVI Tiles to int16 and Compress ======
tif_files = sorted(glob.glob(os.path.join(INPUT_FOLDER, "*.tif")))
print(f"Found {len(tif_files)} TIFF files")
if not tif_files:
    raise RuntimeError("No input files found")
scaled_files = []
processing_errors = []

for idx, tif_path in enumerate(tif_files, 1):
    try:
        base_name = os.path.basename(tif_path)
        output_path = os.path.join(OUTPUT_FOLDER, f"scaled_{base_name}")
        if os.path.exists(output_path) and validate_geotiff(output_path):
            print(f"Skipping existing valid file ({idx}/{len(tif_files)}): {base_name}")
            scaled_files.append(output_path)
            continue
        with rasterio.open(tif_path) as src:
            profile = src.profile.copy()
            profile.update({
                'dtype': 'int16',
                'driver': 'GTiff',
                'compress': 'lzw',
                'predictor': 2,
                'tiled': True,
                'blockxsize': 256,
                'blockysize': 256
            })
            with rasterio.open(output_path, 'w', **profile) as dst:
                for band_idx in range(1, src.count + 1):
                    data = src.read(band_idx)
                    scaled_data = (data * 100).astype('int16')
                    dst.write(scaled_data, band_idx)
        if validate_geotiff(output_path):
            scaled_files.append(output_path)
            print(f"Processed ({idx}/{len(tif_files)}): {base_name}")
        else:
            raise RuntimeError("Output file validation failed")
    except Exception as e:
        processing_errors.append(f"{tif_path}: {str(e)}")
        print(f"Error processing {base_name}: {str(e)}")

if processing_errors:
    raise RuntimeError(f"{len(processing_errors)} errors occurred:\n" + "\n".join(processing_errors))
print(f"Successfully processed {len(scaled_files)}/{len(tif_files)} files")

# ====== Step 2: Build VRT Mosaic (basic, no pixelFunctionType yet) ======
vrt_path = os.path.join(OUTPUT_FOLDER, "mosaic.vrt")
mosaic_path = os.path.join(OUTPUT_FOLDER, MOSAIC_NAME)

print("Building VRT mosaic...")
vrt_options = gdal.BuildVRTOptions(
    resampleAlg='bilinear',
    addAlpha=False,
    VRTNodata=None,
    separate=False
)
vrt_ds = gdal.BuildVRT(vrt_path, scaled_files, options=vrt_options)
vrt_ds = None  # Ensure it is written to disk

# ====== Step 3: Edit VRT XML to Add Averaging for Overlap ======
with open(vrt_path, 'r+') as f:
    vrt_text = f.read()
    if 'pixelFunctionType="average"' not in vrt_text:
        # Add pixelFunctionType="average" in the first VRTRasterBand opening tag
        vrt_text = vrt_text.replace('<VRTRasterBand', '<VRTRasterBand pixelFunctionType="average"', 1)
        f.seek(0)
        f.write(vrt_text)
        f.truncate()
print("Updated VRT to use average pixel function for overlap.")

# ====== Step 4: Translate VRT to Compressed GeoTIFF ======
print("Translating VRT to GeoTIFF mosaic...")
translate_options = gdal.TranslateOptions(
    creationOptions=COMPRESSION_OPTS,
    outputType=gdal.GDT_Int16
)
gdal.Translate(mosaic_path, vrt_path, options=translate_options)

# ====== Step 5: Final Validation ======
if not validate_geotiff(mosaic_path):
    raise RuntimeError("Final mosaic validation failed")
print(f"\nSuccessfully created mosaic: {mosaic_path}")
print(f"Output size: {os.path.getsize(mosaic_path)/1024/1024:.1f} MB")

# ====== Step 6: Clean Up ======
if os.path.exists(vrt_path):
    os.remove(vrt_path)


Found 57 TIFF files
Skipping existing valid file (1/57): NDVI_median_landsat_30m_01_2021.tif
Skipping existing valid file (2/57): NDVI_median_landsat_30m_04_2021.tif
Skipping existing valid file (3/57): NDVI_median_landsat_30m_05_2021.tif
Skipping existing valid file (4/57): NDVI_median_landsat_30m_06_2021-0000000000-0000000000.tif
Skipping existing valid file (5/57): NDVI_median_landsat_30m_06_2021-0000000000-0000032768.tif
Skipping existing valid file (6/57): NDVI_median_landsat_30m_06_2021-0000032768-0000000000.tif
Skipping existing valid file (7/57): NDVI_median_landsat_30m_06_2021-0000032768-0000032768.tif
Skipping existing valid file (8/57): NDVI_median_landsat_30m_08_2021.tif
Skipping existing valid file (9/57): NDVI_median_landsat_30m_09_2021.tif
Skipping existing valid file (10/57): NDVI_median_landsat_30m_10_2021.tif
Skipping existing valid file (11/57): NDVI_median_landsat_30m_11_2021.tif
Skipping existing valid file (12/57): NDVI_median_landsat_30m_12_2021.tif
Skipping exis

## 5 - perplexity - address 0 

In [2]:
import rasterio
import glob
import os
from osgeo import gdal
import numpy as np

# ====== Configuration ======
INPUT_FOLDER = r"G:/My Drive/GEE_Exports"
OUTPUT_FOLDER = r"D:/NDVI_output"
MOSAIC_NAME = "NDVI_median_landsat_30m_2021_USA_mosaic_v2.tif"
COMPRESSION_OPTS = [
    'COMPRESS=LZW',
    'PREDICTOR=2',
    'TILED=YES',
    'BIGTIFF=IF_SAFE',
    'BLOCKXSIZE=256',
    'BLOCKYSIZE=256'
]

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ====== Helper Functions ======
def validate_geotiff(file_path):
    try:
        with rasterio.open(file_path) as src:
            if src.width == 0 or src.height == 0:
                return False
            _ = src.read(1, window=((0,1), (0,1)))
        return os.path.getsize(file_path) > 1024
    except Exception:
        return False

# ====== Step 1: Scale NDVI Tiles to int16 and Compress ======
tif_files = sorted(glob.glob(os.path.join(INPUT_FOLDER, "*.tif")))
print(f"Found {len(tif_files)} TIFF files")
if not tif_files:
    raise RuntimeError("No input files found")
scaled_files = []
processing_errors = []

for idx, tif_path in enumerate(tif_files, 1):
    try:
        base_name = os.path.basename(tif_path)
        output_path = os.path.join(OUTPUT_FOLDER, f"scaled_{base_name}")
        if os.path.exists(output_path) and validate_geotiff(output_path):
            print(f"Skipping existing valid file ({idx}/{len(tif_files)}): {base_name}")
            scaled_files.append(output_path)
            continue
        with rasterio.open(tif_path) as src:
            profile = src.profile.copy()
            profile.update({
                'dtype': 'int16',
                'driver': 'GTiff',
                'compress': 'lzw',
                'predictor': 2,
                'tiled': True,
                'blockxsize': 256,
                'blockysize': 256
            })
            with rasterio.open(output_path, 'w', **profile) as dst:
                for band_idx in range(1, src.count + 1):
                    data = src.read(band_idx)
                    scaled_data = (data * 100).astype('int16')
                    dst.write(scaled_data, band_idx)
        if validate_geotiff(output_path):
            scaled_files.append(output_path)
            print(f"Processed ({idx}/{len(tif_files)}): {base_name}")
        else:
            raise RuntimeError("Output file validation failed")
    except Exception as e:
        processing_errors.append(f"{tif_path}: {str(e)}")
        print(f"Error processing {base_name}: {str(e)}")

if processing_errors:
    raise RuntimeError(f"{len(processing_errors)} errors occurred:\n" + "\n".join(processing_errors))
print(f"Successfully processed {len(scaled_files)}/{len(tif_files)} files")

# ====== Step 2: Build VRT Mosaic (basic, no pixelFunctionType yet) ======
vrt_path = os.path.join(OUTPUT_FOLDER, "mosaic.vrt")
mosaic_path = os.path.join(OUTPUT_FOLDER, MOSAIC_NAME)

print("Building VRT mosaic...")
vrt_options = gdal.BuildVRTOptions(
    resampleAlg='bilinear',
    addAlpha=False,
    VRTNodata=None,
    separate=False
)
vrt_ds = gdal.BuildVRT(vrt_path, scaled_files, options=vrt_options)
vrt_ds = None  # Ensure it is written to disk

# ====== Step 3: Edit VRT XML to Ignore Zeros in Averaging ======
with open(vrt_path, 'r+') as f:
    vrt_text = f.read()
    if 'pixelFunctionType="average"' not in vrt_text:
        # Add pixelFunctionType="average" and the custom pixel function for the first VRTRasterBand
        replacement = '<VRTRasterBand pixelFunctionType="average" subClass="VRTDerivedRasterBand">'
        vrt_text = vrt_text.replace('<VRTRasterBand', replacement, 1)

        # Insert a custom pixel function Python code block in the VRT for excluding 0 values in averaging
        pixel_func = """
        <PixelFunctionLanguage>Python</PixelFunctionLanguage>
        <PixelFunctionCode>
        <![CDATA[
def average_nonzero(in_ar, out_ar, xoff, yoff, xsize, ysize, raster_xsize, raster_ysize, buf_radius, gt, **kwargs):
    arrs = np.array(in_ar)
    arrs = arrs.astype(np.float32)
    # Treat 0 and nodata as invalid; ignore them in average
    valid = (arrs != 0) & np.isfinite(arrs)
    sum_valid = np.where(valid, arrs, 0).sum(axis=0)
    count_valid = valid.sum(axis=0)
    avg = np.where(count_valid != 0, sum_valid / count_valid, 0)
    out_ar[:] = avg
        ]]>
        </PixelFunctionCode>
        """
        # Insert pixel_func after <VRTRasterBand ...>
        insert_idx = vrt_text.find('>') + 1
        vrt_text = vrt_text[:insert_idx] + pixel_func + vrt_text[insert_idx:]

        f.seek(0)
        f.write(vrt_text)
        f.truncate()
print("Updated VRT to ignore 0 in pixel overlap averaging.")

# ====== Step 4: Translate VRT to Compressed GeoTIFF ======
print("Translating VRT to GeoTIFF mosaic...")
translate_options = gdal.TranslateOptions(
    creationOptions=COMPRESSION_OPTS,
    outputType=gdal.GDT_Int16
)
gdal.Translate(mosaic_path, vrt_path, options=translate_options)

# ====== Step 5: Final Validation ======
if not validate_geotiff(mosaic_path):
    raise RuntimeError("Final mosaic validation failed")
print(f"\nSuccessfully created mosaic: {mosaic_path}")
print(f"Output size: {os.path.getsize(mosaic_path)/1024/1024:.1f} MB")

# ====== Step 6: Clean Up ======
if os.path.exists(vrt_path):
    os.remove(vrt_path)


Found 57 TIFF files
Skipping existing valid file (1/57): NDVI_median_landsat_30m_01_2021.tif
Skipping existing valid file (2/57): NDVI_median_landsat_30m_04_2021.tif
Skipping existing valid file (3/57): NDVI_median_landsat_30m_05_2021.tif
Skipping existing valid file (4/57): NDVI_median_landsat_30m_06_2021-0000000000-0000000000.tif
Skipping existing valid file (5/57): NDVI_median_landsat_30m_06_2021-0000000000-0000032768.tif
Skipping existing valid file (6/57): NDVI_median_landsat_30m_06_2021-0000032768-0000000000.tif
Skipping existing valid file (7/57): NDVI_median_landsat_30m_06_2021-0000032768-0000032768.tif
Skipping existing valid file (8/57): NDVI_median_landsat_30m_08_2021.tif
Skipping existing valid file (9/57): NDVI_median_landsat_30m_09_2021.tif
Skipping existing valid file (10/57): NDVI_median_landsat_30m_10_2021.tif
Skipping existing valid file (11/57): NDVI_median_landsat_30m_11_2021.tif
Skipping existing valid file (12/57): NDVI_median_landsat_30m_12_2021.tif
Skipping exis

RuntimeError: Final mosaic validation failed

## 6 - deepseek2

In [1]:
import rasterio
import glob
import os
from osgeo import gdal
import numpy as np

# ====== Configuration ======
INPUT_FOLDER = r"G:/My Drive/GEE_Exports"
OUTPUT_FOLDER = r"D:/NDVI_output"
MOSAIC_NAME = "NDVI_median_landsat_30m_2021_USA_mosaic_v2.tif"
COMPRESSION_OPTS = [
    'COMPRESS=LZW',
    'PREDICTOR=2',
    'TILED=YES',
    'BIGTIFF=IF_SAFE',
    'BLOCKXSIZE=256',
    'BLOCKYSIZE=256',
    'NUM_THREADS=ALL_CPUS'  # Added for faster processing
]
NODATA_VALUE = -32767  # Standard nodata value for int16

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ====== Helper Functions ======
def validate_geotiff(file_path):
    """Validate GeoTIFF by checking dimensions and reading a small portion"""
    try:
        with rasterio.open(file_path) as src:
            if src.width == 0 or src.height == 0:
                return False
            # Try reading first pixel and last pixel
            _ = src.read(1, window=((0, 1), (0, 1)))
            _ = src.read(1, window=((src.height-1, src.height), (src.width-1, src.width)))
        return os.path.getsize(file_path) > 2048  # More reasonable minimum size
    except Exception as e:
        print(f"Validation failed for {file_path}: {str(e)}")
        return False

# ====== Step 1: Scale NDVI Tiles to int16 and Compress ======
tif_files = sorted(glob.glob(os.path.join(INPUT_FOLDER, "*.tif")))
print(f"Found {len(tif_files)} TIFF files")
if not tif_files:
    raise RuntimeError("No input files found")

scaled_files = []
processing_errors = []

for idx, tif_path in enumerate(tif_files, 1):
    try:
        base_name = os.path.basename(tif_path)
        output_path = os.path.join(OUTPUT_FOLDER, f"scaled_{base_name}")
        
        # Skip existing valid files
        if os.path.exists(output_path) and validate_geotiff(output_path):
            print(f"Skipping existing valid file ({idx}/{len(tif_files)}): {base_name}")
            scaled_files.append(output_path)
            continue
            
        with rasterio.open(tif_path) as src:
            # Handle input nodata if exists
            input_nodata = src.nodata if src.nodata is not None else np.nan
            
            # Read and process data
            data = src.read(1)
            
            # Create mask for valid data
            valid_mask = np.ones(data.shape, dtype=bool)
            if np.isnan(input_nodata):
                valid_mask = np.isfinite(data)
            else:
                valid_mask = (data != input_nodata)
            
            # Scale and convert to int16
            scaled_data = np.full(data.shape, NODATA_VALUE, dtype=np.int16)
            scaled_data[valid_mask] = np.clip(
                data[valid_mask] * 100, 
                -10000, 10000
            ).astype(np.int16)
            
            # Update profile
            profile = src.profile.copy()
            profile.update({
                'dtype': 'int16',
                'nodata': NODATA_VALUE,
                'driver': 'GTiff',
                'compress': 'lzw',
                'predictor': 2,
                'tiled': True,
                'blockxsize': 256,
                'blockysize': 256
            })
            
            with rasterio.open(output_path, 'w', **profile) as dst:
                dst.write(scaled_data, 1)
        
        if validate_geotiff(output_path):
            scaled_files.append(output_path)
            print(f"Processed ({idx}/{len(tif_files)}): {base_name}")
        else:
            raise RuntimeError("Output file validation failed")
            
    except Exception as e:
        processing_errors.append(f"{tif_path}: {str(e)}")
        print(f"Error processing {base_name}: {str(e)}")

if processing_errors:
    raise RuntimeError(f"{len(processing_errors)} errors occurred:\n" + "\n".join(processing_errors))
print(f"Successfully processed {len(scaled_files)}/{len(tif_files)} files")

# ====== Step 2: Build VRT Mosaic ======
vrt_path = os.path.join(OUTPUT_FOLDER, "mosaic.vrt")
mosaic_path = os.path.join(OUTPUT_FOLDER, MOSAIC_NAME)

print("Building VRT mosaic...")
vrt_options = gdal.BuildVRTOptions(
    resampleAlg='bilinear',
    VRTNodata=NODATA_VALUE,
    separate=False
)
vrt_ds = gdal.BuildVRT(vrt_path, scaled_files, options=vrt_options)
vrt_ds = None  # Ensure it is written to disk

# ====== Step 3: Edit VRT XML for Smart Averaging ======
print("Updating VRT with pixel function...")
with open(vrt_path, 'r+') as f:
    vrt_text = f.read()
    if 'pixelFunctionType="average"' not in vrt_text:
        # Add pixelFunctionType and subclass
        vrt_text = vrt_text.replace(
            '<VRTRasterBand dataType="Int16" band="1" subClass="VRTRawRasterBand">',
            '<VRTRasterBand dataType="Int16" band="1" subClass="VRTDerivedRasterBand">',
            1
        )
        
        # Insert pixel function definition
        replacement = '''<VRTRasterBand dataType="Int16" band="1" subClass="VRTDerivedRasterBand">
  <PixelFunctionType>average</PixelFunctionType>
  <PixelFunctionLanguage>Python</PixelFunctionLanguage>
  <PixelFunctionCode><![CDATA[
import numpy as np

def average(in_ar, out_ar, xoff, yoff, xsize, ysize, raster_xsize, raster_ysize, buf_radius, gt, **kwargs):
    # Convert to float32 for calculations
    ar = np.array(in_ar).astype(np.float32)
    
    # Create mask for valid pixels (ignore our NODATA_VALUE)
    valid_mask = (ar != {nodata})
    
    # Calculate sum and count of valid pixels
    sum_val = np.sum(np.where(valid_mask, ar, 0), axis=0)
    count_val = np.sum(valid_mask, axis=0)
    
    # Calculate average where we have at least 1 valid pixel
    result = np.where(
        count_val > 0, 
        sum_val / count_val, 
        {nodata}
    )
    
    # Handle potential NaN/Inf just in case
    result[~np.isfinite(result)] = {nodata}
    out_ar[:] = result
]]></PixelFunctionCode>
'''.format(nodata=NODATA_VALUE)
        
        vrt_text = vrt_text.replace(
            '<VRTRasterBand dataType="Int16" band="1" subClass="VRTDerivedRasterBand">',
            replacement,
            1
        )
        
        f.seek(0)
        f.write(vrt_text)
        f.truncate()

print("Updated VRT with pixel function")

# ====== Step 4: Translate VRT to Compressed GeoTIFF ======
print("Translating VRT to GeoTIFF mosaic...")
translate_options = gdal.TranslateOptions(
    creationOptions=COMPRESSION_OPTS,
    outputType=gdal.GDT_Int16,
    noData=NODATA_VALUE
)

# Use warper for better performance
gdal.Warp(
    mosaic_path,
    vrt_path,
    options=gdal.WarpOptions(
        format='GTiff',
        creationOptions=COMPRESSION_OPTS,
        outputType=gdal.GDT_Int16,
        dstNodata=NODATA_VALUE,
        multithread=True,
        warpOptions=['NUM_THREADS=ALL_CPUS']
    )
)

# ====== Step 5: Final Validation ======
if not validate_geotiff(mosaic_path):
    raise RuntimeError("Final mosaic validation failed")
print(f"\nSuccessfully created mosaic: {mosaic_path}")
print(f"Output size: {os.path.getsize(mosaic_path)/1024/1024:.1f} MB")

# ====== Step 6: Clean Up ======
if os.path.exists(vrt_path):
    os.remove(vrt_path)
print("Cleanup complete")

Found 57 TIFF files
Skipping existing valid file (1/57): NDVI_median_landsat_30m_01_2021.tif
Skipping existing valid file (2/57): NDVI_median_landsat_30m_04_2021.tif
Skipping existing valid file (3/57): NDVI_median_landsat_30m_05_2021.tif
Skipping existing valid file (4/57): NDVI_median_landsat_30m_06_2021-0000000000-0000000000.tif
Skipping existing valid file (5/57): NDVI_median_landsat_30m_06_2021-0000000000-0000032768.tif
Skipping existing valid file (6/57): NDVI_median_landsat_30m_06_2021-0000032768-0000000000.tif
Skipping existing valid file (7/57): NDVI_median_landsat_30m_06_2021-0000032768-0000032768.tif
Skipping existing valid file (8/57): NDVI_median_landsat_30m_08_2021.tif
Skipping existing valid file (9/57): NDVI_median_landsat_30m_09_2021.tif
Skipping existing valid file (10/57): NDVI_median_landsat_30m_10_2021.tif
Skipping existing valid file (11/57): NDVI_median_landsat_30m_11_2021.tif
Skipping existing valid file (12/57): NDVI_median_landsat_30m_12_2021.tif
Skipping exis

c:\Users\pc\.conda\envs\geo_env\Lib\site-packages\osgeo\gdal.py:330: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Updating VRT with pixel function...
Updated VRT with pixel function
Translating VRT to GeoTIFF mosaic...

Successfully created mosaic: D:/NDVI_output\NDVI_median_landsat_30m_2021_USA_mosaic_v2.tif
Output size: 5809.2 MB
Cleanup complete
